In [1]:
import pandas as pd
import numpy as np

# Load the CSV files
path = '~/Downloads/'
csv_files = [path + 'file1.csv', path + 'file2.csv']

In [2]:
data_frames = []
last_rows = []

# Iterate over the csv files.
for file in csv_files:

    # Create the dataframe.
    df = pd.read_csv(file)

    # Convert the last row of the last column to list and then np.array.
    column_list_str = df.iloc[-1, -1]
    list_floats = [float(x) for x in column_list_str.strip('[]').split(',')]
    array_floats = np.array(list_floats, dtype=float)

    # Remove the last column (no longer needed).
    df = df.drop('f1_per_class', axis=1)

    # Save the last metric and dataframe.
    last_rows.append(array_floats)
    data_frames.append(df)

# Merge the dataframes based on columns.
df_concat = pd.concat(data_frames, axis=0)

In [3]:
# Group by index (epoch) and compute mean by row.
by_row_index = df_concat.groupby(df_concat.index)
df_means = by_row_index.mean()
df_stds = by_row_index.std()

In [4]:
# Create the col for the last metric.
col_name = 'f1_per_class'

# Compute the mean and std values per column.
avg_final_metric = np.mean(last_rows, axis=0)
std_final_metric = np.std(last_rows, axis=0)

# Add a single value at the last row of the dfs.
df_means[col_name] = ''
df_means.at[df_means.index[-1], col_name] = avg_final_metric
df_stds[col_name] = ''
df_stds.at[df_stds.index[-1], col_name] = std_final_metric

# Fix epoch's column.
df_means['epoch'] = df_means['epoch'].astype('int')
df_stds['epoch'] = df_means['epoch'].astype('int')

In [7]:
# Save the mean and standard deviation dataframes to CSV files.
df_means.to_csv('csv_mean_values.csv', index=False)
df_stds.to_csv('csv_std_values.csv', index=False)